## IO and do notation (Week 2 recap) ##


According to wikibooks even though do notation looks very different from the Haskell code we have seen so far, it is just syntactic sugar for a handful of functions, the most important of them being the (>>=) operator. We could explain how those functions work and then introduce do notation. However, there are several topics we would need to cover before we can give a convincing explanation. 

Jumping in with do right now is a pragmatic shortcut that will allow you to start writing complete programs with IO right away. We will see how do works later in the book, beginning with the Understanding monads chapter. https://en.wikibooks.org/wiki/Haskell/Simple_input_and_output

In [1]:
"Hi, " ++ "test"

"Hi, test"

In [2]:
import System.IO

In [3]:
:t stdin

stdin :: Handle

In [4]:
name <- hGetLine stdin

test


In [5]:
name

"test"

In [6]:
hPutStrLn stdout $ "Hi, " ++ name

Line 1: Use putStrLn
Found:
hPutStrLn stdout
Why not:
putStrLn

Hi, test

In [7]:
:t name

name :: String

In [8]:
:t hGetLine stdin

hGetLine stdin :: IO String

In [9]:
:i (++)

(++) :: [a] -> [a] -> [a] 	-- Defined in ‘GHC.Base’
infixr 5 ++

In [10]:
:i (<-)

: 

In [11]:
:t do

: 

## Actions and pure functions ##

Since the following examples all involve IO, we will refer to the computations/monadic values as actions (as we did in the earlier parts of the book). Of course, do works with any monad; there is nothing specific about IO in how it works.https://en.wikibooks.org/wiki/Haskell/do_notation

In [13]:
name <- hGetLine stdin

test4


In [14]:
hPutStrLn stdout $ "Hi, " ++ name

Line 1: Use putStrLn
Found:
hPutStrLn stdout
Why not:
putStrLn

Hi, test4

In [15]:
:t do name <- hGetLine stdin; hPutStrLn stdout $ "Hi, " ++ name

do name <- hGetLine stdin; hPutStrLn stdout $ "Hi, " ++ name :: IO ()

In [16]:
main = do name <- hGetLine stdin; hPutStrLn stdout $ "Hi, " ++ name

Line 1: Use getLine
Found:
hGetLine stdin
Why not:
getLineLine 1: Use putStrLn
Found:
hPutStrLn stdout
Why not:
putStrLn

In [17]:
:t main

main :: IO ()

In [18]:
main

Hi, test9

## The bind operator and lambda ##

The bind operator (>>=) is a bit more difficult to translate to and from the do notation. (>>=) passes a value, namely the result of an action or function, downstream in the binding sequence. do notation assigns a variable name to the passed value using the <-.

do { x1 <- action1
   ; x2 <- action2
   ; mk_action3 x1 x2 }
The curly braces and the semicolons are optional if every line of code is indented to line up equally (NB: beware the mixing of tabs and spaces in that case; with the explicit curly braces and semicolons indentation plays no part and there's no danger).

x1 and x2 are the results of action1 and action2. If, for instance, action1 is an IO Integer then x1 will be bound to an Integer value. The two bound values in this example are passed as arguments to mk_action3, which creates a third action. The do block is broadly equivalent to the following vanilla Haskell snippet:

action1 >>= (\ x1 -> action2 >>= (\ x2 -> mk_action3 x1 x2 ))
The second argument of the first (leftmost) bind operator (>>=) is a function (lambda expression) specifying what to do with the result of the action passed as the bind's first argument. Thus, chains of lambdas pass the results downstream. The parentheses could be omitted, because a lambda expression extends as far as possible. x1 is still in scope at the point we call the final action maker mk_action3. We can rewrite the chain of lambdas more legibly by using separate lines and indentation:https://en.wikibooks.org/wiki/Haskell/do_notation

In [23]:
main2 = do hGetLine stdin >>= (\x1 -> hPutStrLn stdout $ "Hi, " ++ x1)

Line 1: Use getLine
Found:
hGetLine stdin
Why not:
getLineLine 1: Use putStrLn
Found:
hPutStrLn stdout
Why not:
putStrLn

In [21]:
main2

Hi, test6

In [22]:
:t main2

main2 :: IO ()

## The ''fail'' method ##

Above, we said the snippet with lambdas was "broadly equivalent" to the <code>do</code> block. The translation is not exact because the <code>do</code> notation adds special handling of pattern match failures. When placed at the left of either <code><-</code> or <code>-></code>, <code>x1</code> and <code>x2</code> are patterns being matched. Therefore, if <code>action1</code> returned a <code>Maybe Integer</code> we could write a <code>do</code> block like this...


do { Just x1 <- action1
   ; x2      <- action2
   ; mk_action3 x1 x2 }


...and <code>x1</code> be an <code>Integer</code>. In such a case, what happens if <code>action1</code> returns <code>Nothing</code>? Ordinarily, the program would crash with an non-exhaustive patterns error, just like the one we get when calling <code>head</code> on an empty list. With <code>do</code> notation, however, failures are handled with the <code>fail</code> method for the relevant monad. The <code>do</code> block above translates to:


action1 >>= f
where f (Just x1) = do { x2 <- action2 ; mk_action3 x1 x2 }
      f _         = fail "..." -- A compiler-generated message.


What <code>fail</code> actually does depends on the monad instance. Though it will often rethrow the pattern matching error, monads that incorporate some sort of error handling may deal with the failure in their own specific ways. For instance, <code>Maybe</code> has {{nowrap|1=<code>fail _ = Nothing</code>}}; analogously, for the list monad {{nowrap|1=<code>fail _ = []</code>}}.<ref>This explains why, as we pointed out in the [[Haskell/Pattern_matching#List_comprehensions|"Pattern matching" chapter]], pattern matching failures in list comprehensions are silently ignored.</ref>

The <code>fail</code> method is an artifact of <code>do</code> notation. Rather than calling <code>fail</code> directly, you should rely on automatic handling of pattern match failures whenever you are sure that <code>fail</code> will do something sensible for the monad you are using.
